In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from json import load
import tifffile
from tqdm import tqdm
from skimage.draw import polygon
from matplotlib import rcParams
from glasbey import create_palette

palette = {
    "green": "#558150",
    "beige": "#F1E2C3",
    "brown": "#A7785A",
    "pink": "#F0D6C2",
    "black": "#0E0E0E",
}

rcParams['font.family'] = 'sans-serif'
rcParams['figure.facecolor'] = "#FFFFFF00"
rcParams['axes.facecolor'] = "#FFFFFF00"
rcParams['legend.framealpha'] = 0.2
rcParams['axes.edgecolor'] = palette["black"]
rcParams['axes.labelcolor'] = palette["black"]
rcParams['xtick.color'] = palette["black"]
rcParams['ytick.color'] = palette["black"]
rcParams['text.color'] = palette["black"]
rcParams['axes.titlecolor'] = palette["black"]

In [3]:
base = Path(r"D:\Tracking\NucleiTracking\data\interim\lightsheet\2025_03_18_trk\uv_unwrap")
# raw = tifffile.imread(list(base_path.glob("*.tif"))[1])

In [14]:
from skimage.measure import regionprops
from scipy.spatial.distance import cdist
import napari

locs = tifffile.imread(base / f"all_locs.tif")
masks = tifffile.imread(base / f"all_vals2_substack_cp_masks.tif")
vals = tifffile.imread(base / f"all_vals2_substack.tif")

centroids = []

for t, (maskslice, locslice) in tqdm(enumerate(zip(masks, locs))): 

    frame_centroids = []
    
    props = regionprops(maskslice, locslice)
    
    for prop in props:
        ct = [prop.centroid[0], prop.centroid[1]]
        round_ct = [int(np.round(c)) for c in prop.centroid_local]
        
        ct = np.median(np.median(prop.image_intensity[round_ct[0] - 1:round_ct[0] + 2, round_ct[1] - 1:round_ct[1] + 2], axis=0), axis=0)
        
        frame_centroids.append([t, *ct])
                
    
    centroids.extend(frame_centroids)
                
centroids = np.array(centroids)

viewer = napari.Viewer()
pts = viewer.add_points(centroids, shading="spherical")

napari.run()

71it [00:10,  7.06it/s]


In [15]:
import pandas as pd
cdf = {"frame": centroids[:, 0], "z": centroids[:, 1], "y": centroids[:, 2], "x": centroids[:, 3]}
df = pd.DataFrame(cdf)
df["frame"] = np.array(centroids[:, 0], dtype=int)
df.to_csv(base / "spots.csv")